In [183]:
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession, DataFrame

spark = (SparkSession.builder
            .master("local[2]")
            .config("spark.driver.bindAddress", "localhost")
            .config("spark.driver.port", "8080")
            .config("spark.driver.memory", "2g")
            .config("spark.driver.host", "localhost")
            .config("spark.executor.memory", "3g" )
            .config("spark.executor.cores", "5" )
            .config("spark.dynamicAllocation.enabled", "true" )
            .config("spark.default.parallelism", "2" )
            .config("spark.shuffle.io.retryWait", "2000ms" )
            .config("spark.shuffle.io.maxRetries", "2" )
            .getOrCreate())

In [184]:
df = spark.read.csv("data/db/db.csv", sep="!", header=True)

def get_df_with_suffix(df, suffix):
    return df.select([fn.col(c).alias(f"{c}{suffix}") for c in df.columns])

def merge_on(suffix, df1, df2): 
    df_new = get_df_with_suffix(df2, suffix=suffix)
    return df1.join(df_new, df1[f"key{suffix}"]==df_new[f"pkey{suffix}"], "inner")


In [185]:
train_df = spark.read.csv("data/train.csv", header=True)

train_df = train_df.drop(*["id", "partition"])

colsToDrop = [
    "_c0",
    "paddress",
    "ppublisher",
    "pseries",
    "pbooktitlefull_id",
    "pjournalfull_id",
    "peditor",
    "pbooktitle_id",
    "partition"
]
df = df.drop(*colsToDrop)


# train_df = merge_on("1", train_df, df)

# train_df.take(5)
# train = train_df[[
#     "label",
#     "pyear1", "pkey1", "clean_author1", "clean_title1", "ptype_id1", "pjournal_id1", "pbooktitle_id1", "pjournalfull_id1", "pbooktitlefull_id1", 
#     "pyear2", "pkey2", "clean_author2", "clean_title2", "ptype_id2", "pjournal_id2", "pbooktitle_id2", "pjournalfull_id2", "pbooktitlefull_id2"
#     ]]

df.take(2)

[Row(pyear='2007.0', pid='180843', pkey='conf/dft/SemiaoRVSTT07', ptype_id='1', pjournal_id='0', clean_author='Jorge Semião|Juan J. Rodríguez-Andina|Fabian Vargas|Marcelino Bicho Dos Santos|Isabel C. Teixeira|João Paulo Teixeira', clean_title='Improving the Tolerance of Pipeline Based Circuits to Power Supply or Temperature Variations.'),
 Row(pyear='2007.0', pid='162991', pkey='conf/dagstuhl/Caire07', ptype_id='2', pjournal_id='0', clean_author='Patrice Caire', clean_title='A Normative Multi-Agent Systems Approach to the Use of Conviviality for Digital Cities.')]

In [186]:
train_df = merge_on("1", train_df, df).repartition(2)
train_df = merge_on("2", train_df, df)

In [187]:
train_df.take(2)
# get_df_with_suffix(df, suffix="2").take(2)

22/03/21 13:44:27 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Failed to connect to katrins-mbp:51784
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:288)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:218)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:126)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:154)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:133)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:146)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManag

Py4JJavaError: An error occurred while calling o4321.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 262.0 failed 1 times, most recent failure: Lost task 0.0 in stage 262.0 (TID 367) (katrins-mbp executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2454)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2403)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2402)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2402)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed

In [13]:
def jaccard_similarity(a, b):
    # convert to set
    a = set(a)
    b = set(b)
    # calucate jaccard similarity
    j = float(len(a.intersection(b))) / len(a.union(b))
    return j

# TODO translate to spark still
train.loc[:, 'jaccard_author'] = train[["clean_author1", "clean_author2"]].apply(lambda x: jaccard_similarity(x["clean_author1"], x["clean_author2"]), axis=1)
train.loc[:, 'jaccard_title'] = train[["clean_title1", "clean_title2"]].apply(lambda x: jaccard_similarity(x["clean_title1"], x["clean_title2"]), axis=1)
train.head()

/Users/katringrunert/Projects/Uni/big_data/project/pipeline/bigd/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,label,pyear1,pkey1,clean_author1,clean_title1,ptype_id1,pjournal_id1,pbooktitle_id1,pjournalfull_id1,pbooktitlefull_id1,...,pkey2,clean_author2,clean_title2,ptype_id2,pjournal_id2,pbooktitle_id2,pjournalfull_id2,pbooktitlefull_id2,jaccard_author,jaccard_title
0,False,2007.0,conf/prib/AhmedF07,Said Hassan Ahmed|Tor Flå,Estimation of Evolutionary Average Hydrophobic...,0,0.0,0,0.0,0.0,...,journals/jcc/PatraS09,Jagdish Chandra Patra|Onkar Singh,Artificial neural networks-based approach to d...,0,7.0,0,7.0,0.0,0.384615,0.628571
1,True,1995.0,conf/vlsid/ChenCC95,Yung-Yuan Chen|Ching-Hwa Cheng|Jwu-E Chen,An efficient switching network fault diagnosis...,0,0.0,1,0.0,1.0,...,journals/tcad/LuoCWCCW08,Pei-Wen Luo|Jwu-E Chen|Chin-Long Wey|Liang-Chi...,Impact of Capacitance Correlation on Yield Enh...,0,220.0,5,220.0,5.0,0.666667,0.611111
2,True,1988.0,conf/prozess/Sun88,Anwendung graphischer Darstellungen im Rahmen ...,Z. Sun,1,0.0,2,0.0,2.0,...,conf/isnn/SunZLCS07,Z. Sun|M. J. Zhang|Xiao H. Liao|Wenchuan Cai|Y...,Neuro-Adaptive Formation Control of Multi-Mobi...,1,0.0,436,0.0,436.0,0.333333,0.111111
3,False,2004.0,conf/pricai/BeaumontTSM04,Matthew Beaumont|John Thornton|Abdul Sattar|Mi...,Solving Over-Constrained Temporal Reasoning Pr...,2,0.0,3,0.0,3.0,...,conf/icip/SattarAS08,Abdul Sattar 0003|Yasser Aidarous|Renaud Séguier,GAGM-AAM: A genetic optimization with Gaussian...,2,0.0,365,0.0,365.0,0.483871,0.513514
4,False,2005.0,conf/aiide/BourneS05,Owen Bourne|Abdul Sattar,Applying Constraint Weighting to Autonomous Ca...,1,0.0,337,0.0,337.0,...,conf/icip/SattarAS08,Abdul Sattar 0003|Yasser Aidarous|Renaud Séguier,GAGM-AAM: A genetic optimization with Gaussian...,2,0.0,365,0.0,365.0,0.560000,0.548387


## Feature Prep

In [19]:
required_features = ['pyear2',
                    'pyear1',
                    'jaccard_author',
                    'jaccard_title',
                    'pjournal_id2',
                    'pjournal_id1',
                    'pyear1',
                    'pyear2'
                   ]

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=required_features, outputCol='features')

transformed_data = assembler.transform(spark_df)
# transformed_data.show()
transformed_data = transformed_data.withColumn("label", fn.col("label").cast("Integer"))

## Training

In [20]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='label', 
                            featuresCol='features',
                            maxDepth=5)
model = rf.fit(transformed_data)
# rf_predictions = model.transform(test_data)

22/03/21 08:25:59 WARN TaskSetManager: Stage 9 contains a task of very large size (3003 KiB). The maximum recommended task size is 1000 KiB.
22/03/21 08:26:00 WARN TaskSetManager: Stage 10 contains a task of very large size (3003 KiB). The maximum recommended task size is 1000 KiB.
22/03/21 08:26:00 WARN TaskSetManager: Stage 16 contains a task of very large size (3003 KiB). The maximum recommended task size is 1000 KiB.
22/03/21 08:26:03 WARN TaskSetManager: Stage 39 contains a task of very large size (3003 KiB). The maximum recommended task size is 1000 KiB.
22/03/21 08:26:03 WARN TaskSetManager: Stage 40 contains a task of very large size (3003 KiB). The maximum recommended task size is 1000 KiB.


## Evaluation

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'Outcome', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', multi_evaluator.evaluate(rf_predictions))